# CS-2376 Apriori Algorithm
### Team Members
1. Jenish Raj Bajracharya
2. Nidup Dorji
3. Satpreet Makhija


In [79]:
import pandas as pd
import numpy as np
import operator
from itertools import combinations
import csv

## Data Pre-processing
1. Reading Data
2. Getting unique dataset

In [2]:
dataframe = pd.read_csv("dataset/tesco2.csv", header=None)
# dataframe.head()
dataframe = dataframe.values

data = set()
for items in dataframe:
    for sub_item in items:
        data.add(sub_item)

min_support = int(input("Enter support "))
min_confidence = float(input("Enter confidence level "))
# unique_data = list(data)
# print(unique_data)



In [3]:

def get_frequent_1_set(min_support, dataframe, data):
    temp_data = dict()
    itemset = data
    for i in itemset:
        count = 0
        for d in dataframe:
            if i in d:
                count += 1
        temp_data[i] = count
        if temp_data[i] < min_support:
            temp_data.pop(i)
    # return list(temp_data.keys())
    return sorted(temp_data.keys(), key=operator.itemgetter(0))


# def get_item_combination():
F1 = get_frequent_1_set(min_support, dataframe, data) 
# print(F1)

In [4]:

def generate_candidate(array, size): # L1 is a list from get_freauent_set
    return sum([list(map(set, combinations(array, size)))], [])
  
# print(generate_candidate(F1,4))


In [5]:

# Lk_1 = [['apple', 'beer'], ['apple', 'rice'], ['beer', 'chicken'], ['beer', 'milk'], ['beer', 'rice'], ['chicken', 'rice'], ['milk', 'rice']]
def apriori_gen(Lk_1, k):
    ck = [] 
    for i in Lk_1:
       
        for j in Lk_1:
           
            index = 0
            buffer = []
            if i == j:
                pass
            else:
                while k-1 > index:
                    if i[index] == j[index]:
                        index += 1
                    else:
                        break
                else:
                    if i[k-1] > j[k-1]:
                        
                        for item in i:
                            buffer.append(item)
                        buffer.append(j[k-1])
                        check = generate_candidate(buffer, k)
                        count = 0
                        for c in check:
                            sorting = sorted(c)
                            if list(sorting) in Lk_1:
                                count += 1
                        if count == len(check):
                            ck.append(sorted(buffer))
                           
    return ck
                        



In [106]:
def get_frequent_set(min_support, dataframe):
    F1 = get_frequent_1_set(min_support, dataframe, data)
    Fk_1 = []
    for item in F1:
        buffer = []
        buffer.append(item)
        Fk_1.append(buffer)
    print("                                              Frequent 1 Itemset")
    print(Fk_1)
    print("\n")

    all_frequent_item = dict()
    k = 1
    while len(Fk_1) != 0:
     
        ck = apriori_gen(Fk_1, k)
        Fk = []

        for c in ck:
            count = 0
            candidate = set(c)
            for d in dataframe:
                if candidate.issubset(d):
                    count += 1
            if count >= min_support:
                sorting = sorted(candidate)
                Fk.append(sorting)
                all_frequent_item[tuple(c)] = count
        if Fk == []:
            print("                                              Frequent {} Itemset".format(k+1))
            print("None")
            print("\n")
        else:
            print("                                              Frequent {} Itemset".format(k+1))
            print(Fk)
            print("\n")
        Fk_1 = []
        k += 1
      
        for item in Fk:
            Fk_1.append(item)
        
    return(all_frequent_item)




In [109]:
def get_association_rule():
    frequent_item = get_frequent_set(min_support, dataframe)
    count = 0
    rule_count = 0
    print("                                              Association Rules")
    for item in frequent_item:
        count = 1
        # print("item : {}".format(item))
        sup_b = frequent_item[item]
        # print(item)
        while count < len(item):
            combo = generate_candidate(item,count)  # give us the possible combinations for item
            # print("combo : {}".format(combo))
            count = count + 1
            for c in combo:
                a = []
                sup_a = 0
                b = []
                
                for i in c:
                    a.append(i)
                for data in dataframe:
                        if set(a).issubset(data):
                            sup_a = sup_a + 1 
                 
                confidence = 100*sup_b/sup_a
                if confidence >= min_confidence:
                    for sub_item in item: 
                        # print("a: {}".format(a))
                        # print("subitem: {}".format(sub_item))
                        if sub_item not in a:
                            b.append(sub_item)
                        # print("b : {}".format(b))
                    rule_count = rule_count + 1
                    
                    print("================== Rule {} =============================".format(rule_count))
                    print (" {} ==> {}".format(a,b))
                    print("Support = {}".format(sup_b))
                    print("Condidence = {}%".format(confidence))
                    print("\n")
                   
                   

In [110]:
get_association_rule()

                                              Frequent 1 Itemset
[['apple'], ['beer'], ['chicken'], ['fanta'], ['google'], ['mango'], ['milk'], ['rice']]


                                              Frequent 2 Itemset
[['apple', 'beer'], ['beer', 'chicken'], ['apple', 'fanta'], ['apple', 'mango'], ['fanta', 'mango'], ['beer', 'milk'], ['apple', 'rice'], ['beer', 'rice'], ['chicken', 'rice'], ['milk', 'rice']]


                                              Frequent 3 Itemset
[['apple', 'fanta', 'mango'], ['apple', 'beer', 'rice'], ['beer', 'chicken', 'rice'], ['beer', 'milk', 'rice']]


                                              Frequent 4 Itemset
None


                                              Association Rules
================== Rule 1 =============================
 ['apple'] ==> ['beer']
Support = 3
Condidence = 50.0%


================== Rule 2 =============================
 ['beer'] ==> ['apple']
Support = 3
Condidence = 50.0%


================== Rule 3 ===============